In [1]:
from matplotlib.path import Path
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import random
import time
import json
import cplex._internal._procedural
import cplex
import pandas as pd
from datetime import datetime  
from datetime import timedelta 
import math
from mpl_toolkits import mplot3d
%matplotlib inline
import pickle

import plotly.plotly as py
from heapq import heappush, heappop
from itertools import count

import networkx as nx
import os


In [2]:
path = '/Users/mj/Desktop/INFORMS_CODE_FILE/network design code/Files_to_solve' ## Please set up a directory for the code file.

vdc_index_dic = pickle.load(open(path +'/vdc_index_all_dic.dump', 'rb'))
plant_index_dic = pickle.load(open(path + '/plant_index_dic.dump','rb'))

index_vdc_dic = pickle.load(open(path +'/index_vdc_dic.dump', 'rb'))
index_plant_dic = pickle.load(open(path +'/index_plant_dic.dump', 'rb'))

dist_mat_all = pickle.load(open(path +'/dist_mat_9361.dump','rb'))
location_index_all_dic = pickle.load(open(path +'/location_index_9361.dump', 'rb'))
existing_vdc_dic = pickle.load(open(path +'/Existing_VDC_index_dic.dump', 'rb'))

dealer_index_set_dic = pickle.load(open(path +'/dealer_index_set_dic.dump','rb'))
index_dealer_set_dic = pickle.load(open(path +'/index_dealer_set_dic.dump','rb'))

In [3]:
def Assignment_formulate(plant_index_dic,vdc_index_dic,dealer_index_set_dic,existing_vdc_dic,dist_mat_all,location_index_all_dic):
    
    P = [plant for plant in range(len(plant_index_dic))]

    VDC = [vdc for vdc in range(len(vdc_index_dic))]

    D = [dealer for dealer in range(len(dealer_index_set_dic))]

    existing_vdc = [existing_vdc_dic[vdc] for vdc in existing_vdc_dic]
    
    T = {}

    for i in vdc_index_dic:
        T[vdc_index_dic[i][0]] = {}
        for j in dealer_index_set_dic:
            T[vdc_index_dic[i][0]][dealer_index_set_dic[j]] = dist_mat_all[location_index_all_dic[i]][location_index_all_dic[str(j)]]

    cos_theta = np.zeros((len(VDC),len(D),len(P)))

    for p in plant_index_dic:
        for i in vdc_index_dic:
            for j in dealer_index_set_dic:
                if dist_mat_all[location_index_all_dic[p]][location_index_all_dic[str(j)]] > 0.01 and dist_mat_all[location_index_all_dic[i]][location_index_all_dic[str(j)]] > 0.01 and dist_mat_all[location_index_all_dic[i]][location_index_all_dic[p]] > 0.01 :

                    a=dist_mat_all[location_index_all_dic[p]][location_index_all_dic[str(j)]]
                    b=dist_mat_all[location_index_all_dic[i]][location_index_all_dic[str(j)]]
                    c=dist_mat_all[location_index_all_dic[i]][location_index_all_dic[p]]

                    cos_theta[vdc_index_dic[i][0]][dealer_index_set_dic[j]][plant_index_dic[p]] = ((a**2)+(b**2)-(c**2))/(2*a*b)

                else:
                    cos_theta[vdc_index_dic[i][0]][dealer_index_set_dic[j]][plant_index_dic[p]] = 1
                    
    cpx = cplex.Cplex()

    objs = [
        (assign_alpha*T[i][j])-(cos_theta[i][j][p]*T[i][j])+(assign_alpha*cos_theta[i][j][p]*T[i][j])
        for j in D
        for i in VDC
        for p in P
    ]

    types=["B"] * len(objs)

    names=[f'x_{i}_{j}_{p}'
        for j in D
        for i in VDC
        for p in P
    ]

    cpx.variables.add(
        obj=objs,
        types=types,
        names=names
    )

    cpx.variables.add(
        obj=[1]*len(VDC),
        types=["B"]*len(VDC),
        names=[f'y_{i}' for i in VDC]
    )

    cpx.linear_constraints.add(
        lin_expr=[
            cplex.SparsePair(
                [f'y_{i}' for i in VDC],([1]*len(VDC))
                )
        ],
        senses=['E'],
        rhs=[Q],
        names=['vdc_open_num_limit_1'])

    cpx.linear_constraints.add(
        lin_expr=[
            cplex.SparsePair(
                [f'y_{i}'],[1]
                )
            for i in existing_vdc
        ],
        senses=['E']*len(existing_vdc),
        rhs=[1]*len(existing_vdc),
        names=[f'vdc_open_num_limit_2_{i}' for i in existing_vdc]
    )

    cpx.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
            [f'y_{i}'] + [f'x_{i}_{j}_{p}'], [-1] + [1]
        )
        for i in VDC
        for j in D
        for p in P
    ],
        senses=['L'] * len(D) * len(VDC) * len(P),
        rhs=[0] * len(D) * len(VDC) * len(P),
        names=[f'open_{i}_{j}_{p}' for i in VDC for j in D for p in P]
    )

    cpx.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
            [f'x_{i}_{j}_{p}' for i in VDC],  ([1]*len(VDC))
        )
        for j in D
        for p in P
    ],
        senses=['E'] * len(D) * len(P),
        rhs=[1] * len(D) * len(P),
        names=[f'assignment_{j}_{p}' for j in D for p in P]
    )
    
    return cpx,P,VDC,D

def solve_assignment(P,VDC,D,cpx,cpu=4,timelimit=3600):
    
    start_time= time.time()
    cpx.parameters.threads.set(cpu)
    cpx.parameters.timelimit.set(timelimit)
    # cpx.write("190117.lp")
    cpx.solve()
    num_bnb_nodes = cplex._internal._procedural.getnodecnt(cpx._env._e, cpx._lp)
    num_gap = cplex._internal._procedural.getmiprelgap(cpx._env._e, cpx._lp)    
    sol = cpx.solution
    print("Solution status = ", sol.get_status(), ":", end=' ')
    assign_solve_time = time.time() - start_time

    plant_dealer_finalvdc_dic = {
        
        (index_plant_dic[p],index_dealer_set_dic[j]): index_vdc_dic[i]
        for j in D
        for i in VDC
        for p in P
        if sol.get_values('x_%d_%d_%d' %(i,j,p)) > 0.9
    }

    pickle.dump(plant_dealer_finalvdc_dic,open(path + f'/plant_dealer_finalvdc_dic_{Q}_{assign_alpha}.dump',"wb"))
    pickle.dump(assign_solve_time,open(path + f'/assign_solve_time_{Q}_{assign_alpha}.dump',"wb"))

    openvdc = []

    for i in VDC:
        if sol.get_values('y_%d' %(i)) > 0:
            openvdc.append(i)
            
    pickle.dump(openvdc,open(path + f'/openvdc_{Q}_{assign_alpha}.dump',"wb"))


In [4]:
Qs = [44]
assign_alphas = [0.65]

for Q in Qs:

    for assign_alpha in assign_alphas:

        cpx,P,VDC,D = Assignment_formulate(plant_index_dic,vdc_index_dic,dealer_index_set_dic,existing_vdc_dic,dist_mat_all,location_index_all_dic)
        solve_assignment(P,VDC,D,cpx,cpu=4,timelimit=3600)


CPXPARAM_TimeLimit                               3600
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
Tried aggregator 1 time.
MIP Presolve eliminated 4180593 rows and 4143664 columns.
All rows and columns eliminated.
Presolve time = 4.54 sec. (2879.25 ticks)

Root node processing (before b&c):
  Real time             =    5.93 sec. (3623.01 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    5.93 sec. (3623.01 ticks)
Solution status =  101 : 